<a href="https://colab.research.google.com/github/srivignsh001-lgtm/AIT_LAB_26292/blob/main/AIT_LAB_26292.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
N = 4 # Size of the grid
# Function to print the board
def print_board(board):
    for row in board:
        print(" ".join('G' if col else '.' for col in row))
    print()

# Check if it's safe to place a guard at board[row][col]
def is_safe(board, row, col):
    # Check vertical up
    for i in range(row):
        if board[i][col] == 1:
            return False
    # Check upper left diagonal
    i, j = row - 1, col - 1
    while i >= 0 and j >= 0:
        if board[i][j] == 1:
            return False
        i -= 1
        j -= 1
    # Check upper right diagonal
    i, j = row - 1, col + 1
    while i >= 0 and j < N:
        if board[i][j] == 1:
            return False
        i -= 1
        j += 1
    return True

# Backtracking function
def solve_n_queens(board, row):
    if row == N:
        print_board(board)
        return True
    res = False
    for col in range(N):
        if is_safe(board, row, col):
            board[row][col] = 1
            res = solve_n_queens(board, row + 1) or res
            board[row][col] = 0  # Backtrack
    return res

# Initialize the board
board = [[0 for _ in range(N)] for _ in range(N)]
print("Valid configurations for placing 4 guards:\n")
solve_n_queens(board, 0)


Valid configurations for placing 4 guards:

. G . .
. . . G
G . . .
. . G .

. . G .
G . . .
. . . G
. G . .



True

task8

In [2]:
# Grid coordinates: (x, y)
toy_position = (2, 2)        # Toy is placed at center ceiling
box_position = [0, 0]        # Initial box position
puppy_position = [0, 1]      # Puppy starts in corner
has_toy = False
on_box = False

# Action log
actions = []

# Define room size (optional for visualization)
room_size = 3  # 3x3 grid


# ---------- Helper functions ---------- #
def walk(to):
    """Move the puppy to a location."""
    global puppy_position
    actions.append(f"Walk from {tuple(puppy_position)} to {to}")
    puppy_position[0], puppy_position[1] = to


def push_box(to):
    """Push the box to a location (puppy moves with box)."""
    global puppy_position, box_position
    actions.append(f"Push box from {tuple(box_position)} to {to}")
    box_position[0], box_position[1] = to
    puppy_position[0], puppy_position[1] = to


def climb_box():
    """Puppy climbs onto the box."""
    global on_box
    actions.append("Climb on box")
    on_box = True


def grab_toy():
    """Puppy grabs the toy."""
    global has_toy
    actions.append("Grab the toy")
    has_toy = True


# ---------- Goal Stack Planning ---------- #
stack = ['HasToy']

while stack:
    goal = stack.pop()

    if goal == 'HasToy':
        if has_toy:
            continue
        if puppy_position == box_position == list(toy_position) and on_box:
            grab_toy()
        else:
            stack.append('HasToy')                      # retry after subgoals
            stack.append('OnBox')
            stack.append(('MoveBoxUnderToy', toy_position))

    elif goal == 'OnBox':
        if on_box:
            continue
        if puppy_position == box_position:
            climb_box()
        else:
            stack.append('OnBox')
            stack.append(('WalkTo', tuple(box_position)))

    elif isinstance(goal, tuple) and goal[0] == 'MoveBoxUnderToy':
        target = goal[1]
        if box_position == list(target):
            continue
        stack.append(('MoveBoxUnderToy', target))
        stack.append(('WalkTo', tuple(box_position)))
        stack.append(('PushBoxTo', target))

    elif isinstance(goal, tuple) and goal[0] == 'WalkTo':
        target = goal[1]
        if puppy_position == list(target):
            continue
        walk(target)

    elif isinstance(goal, tuple) and goal[0] == 'PushBoxTo':
        target = goal[1]
        if box_position == list(target):
            continue
        push_box(target)


# ---------- Display the steps ---------- #
print("Plan to achieve goal:")
for step in actions:
    print("-", step)

# ---------- Final state ---------- #
print("\nFinal State:")
print("Puppy Position:", tuple(puppy_position))
print("Box Position:", tuple(box_position))
print("On Box:", on_box)
print("Has Toy:", has_toy)



Plan to achieve goal:
- Push box from (0, 0) to (2, 2)
- Walk from (2, 2) to (0, 0)
- Walk from (0, 0) to (2, 2)
- Climb on box
- Grab the toy

Final State:
Puppy Position: (2, 2)
Box Position: (2, 2)
On Box: True
Has Toy: True


task7.2


In [3]:
state = {
    'monkey_location': 'door',
    'box_location': 'window',
    'monkey_on_box': False,
    'monkey_has_banana': False
}
goal_stack = ['has(monkey, banana)']
actions = []
def walk(x, y):
    state['monkey_location'] = y
    actions.append(f"Walk from {x} to {y}")
def push(x, y):
    state['monkey_location'] = y
    state['box_location'] = y
    actions.append(f"Push box from {x} to {y}")
def climb_up():
    state['monkey_on_box'] = True
    actions.append("Climb up the box")
def grasp():
    state['monkey_has_banana'] = True
    actions.append("Grasp the banana")
while goal_stack:
    goal = goal_stack.pop()
    if goal == 'has(monkey, banana)':
        if not state['monkey_has_banana']:
            goal_stack.append('perform(grasp)')
            goal_stack.append('at(monkey, middle)')
            goal_stack.append('on(monkey, box)')
            if goal == 'perform(grasp)':
                if state['monkey_location'] == 'middle' and state['monkey_on_box']:
                    grasp()
            elif goal == 'on(monkey, box)':
                if state['monkey_location'] != state['box_location']:
                    goal_stack.append('perform(climb_up)')
                else:
                    climb_up()
            elif goal == 'at(monkey, middle)':
                if goal.startswith('at(monkey,'):
                    target = goal.split(',')[1].strip(' )')
                    if state['monkey_location'] != target:
                        if state['monkey_location'] == state['box_location']:
                            push(state['box_location'], target)
                        else:
                            walk(state['monkey_location'], target)
                else:
                    walk(state['monkey_location'], target)

print("Monkey-Banana Planning Actions:")
for step in actions:
    print(" -", step)
print("Final State:")
for key, value in state.items():
    print(f"{key}: {value}")


Monkey-Banana Planning Actions:
Final State:
monkey_location: door
box_location: window
monkey_on_box: False
monkey_has_banana: False


task 7.1
